In [1]:
using Pkg
Pkg.activate("..")
using Revise

  Activating environment at `~/.julia/dev/FUSE/Project.toml`


In [2]:
using FUSE

┌ Info: Precompiling FUSE [e64856f0-3bb8-4376-b4b7-c03396503992]
└ @ Base loading.jl:1317
Progress:   0%|                           |  ETA: 0:04:16 ( 0.29  s/it)ERROR: LoadError: ArgumentError: core_profiles.profiles_1d[:].zeff_fit.source IMAS STR_1D has not been mapped to Julia data type
Stacktrace:
 [1] top-level scope
   @ ~/.julia/dev/FUSE/src/FUSE.jl:74
 [2] include
   @ ./Base.jl:386 [inlined]
 [3] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt64}}, source::Nothing)
   @ Base ./loading.jl:1213
 [4] top-level scope
   @ none:1
 [5] eval
   @ ./boot.jl:360 [inlined]
 [6] eval(x::Expr)
   @ Base.MainInclude ./client.jl:446
 [7] top-level scope
   @ none:1
in expression starting at /Users/meneghini/.julia/dev/FUSE/src/FUSE.jl:3


LoadError: Failed to precompile FUSE [e64856f0-3bb8-4376-b4b7-c03396503992] to /Users/meneghini/.julia/compiled/v1.6/FUSE/jl_5lKJAE.

In [6]:
data=FUSE.dd();

data.core_profiles.profiles_1d[1].grid.rho_tor_norm=range(0,1,length=10)
data.core_profiles.profiles_1d[1].electrons.density=(1.0.-data.core_profiles.profiles_1d[1].grid.rho_tor_norm).^2

LoadError: type dd has no field core_profiles